## Ridge Regression Analysis of Baseline C-Peptide AUC Levels

This notebook performs ridge regression to predict log-transformed baseline 4-hour C-peptide AUC from the 3×3 model features. We will:
- Load and process the tidy baseline AUC datasets from SDY569, SDY797, and SDY1737.
- Train and evaluate local, federated, and retrained models using ridge regression.
- Use median MSE and interquartile range (IQR) to compare performance.
- Visualize predicted vs. actual outcomes.

Each dataset will be split into train/test partitions.


In [1]:
import os

# ========== STEP 0: Set working directory (for running locally on laptop) =========
os.getcwd()
os.chdir("/Users/adeslatt/Scitechcon Dropbox/Anne DeslattesMays/projects/oadr-autoantibody")
os.getcwd()

'/Users/adeslatt/Scitechcon Dropbox/Anne DeslattesMays/projects/oadr-autoantibody'

In [3]:
# Setup

import os
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import load_model

# Set random seed for reproducibility
np.random.seed(42)


In [4]:
# Load feature and label data

# Paths for input features
features_paths = {
    "SDY569": "data/SDY569_tidy.csv",
    "SDY797": "data/SDY797_tidy.csv",
    "SDY1737": "data/SDY1737_tidy.csv"
}

# Paths for c-peptide AUC labels
labels_paths = {
    "SDY569": "data/SDY569_cpeptide_auc_tidy.csv",
    "SDY797": "data/SDY797_cpeptide_auc_tidy.csv",
    "SDY1737": "data/SDY1737_cpeptide_auc_tidy.csv"
}

# Load feature and label DataFrames
feature_dfs = {study: pd.read_csv(path) for study, path in features_paths.items()}
label_dfs = {study: pd.read_csv(path) for study, path in labels_paths.items()}

# Quick check
for study in feature_dfs:
    print(f"{study} features: {feature_dfs[study].shape}, labels: {label_dfs[study].shape}")


SDY569 features: (30, 6), labels: (10, 6)
SDY797 features: (245, 6), labels: (49, 6)
SDY1737 features: (64, 6), labels: (16, 5)


In [37]:
# Check unique properties in SDY569 features
print("Unique properties in SDY569 features:")
print(feature_dfs["SDY569"]["Property"].unique())


Unique properties in SDY569 features:
['gad65' 'ia_2ic' 'miaa']


In [38]:
# Check unique properties in SDY797 features
print("Unique properties in SD797 features:")
print(feature_dfs["SDY797"]["Property"].unique())


Unique properties in SD797 features:
['GAD65' 'IA_2ic' 'ICA' 'MIAA' 'Zn_T8']


In [39]:
# Check unique properties in SDY1737 features
print("Unique properties in SDY1737 features:")
print(feature_dfs["SDY1737"]["Property"].unique())


Unique properties in SDY1737 features:
['GAD65' 'IA_2ic' 'MIAA' 'Zn_T8']


In [27]:
# Normalize subject ID column and merge features with labels
merged_dfs = {}

for study in feature_dfs:
    features = feature_dfs[study].copy()
    labels = label_dfs[study].copy()

    # Rename Accession → Subject_ID for consistency
    if "Accession" in features.columns:
        features = features.rename(columns={"Accession": "Subject_ID"})

    # Merge on Subject_ID
    merged = pd.merge(features, labels, on="Subject_ID", how="inner")
    merged_dfs[study] = merged

    print(f"{study} merged data: {merged.shape[0]} rows")
    display(merged.head(1))


SDY569 merged data: 30 rows


,Subject_ID,Sex,Age_Group,Property,Value,Study_x,Study_y,Participant_ID,Visit_Label,C_Peptide_AUC_4Hrs,Units
0,SUB151307,Female,8-12,gad65,0.04,SDY569,SDY569,ITN007AI_195962,Baseline,0.5625,NG/ML


SDY797 merged data: 245 rows


,Subject_ID,Sex,Age_Group,Property,Value,Study_x,Study_y,Participant_ID,Visit_Label,C_Peptide_AUC_4Hrs,Units
0,SUB168890,Male,8-12,GAD65,1.0,SDY797,SDY797,T1DAL_137962,-1,0.860031,NG/ML


SDY1737 merged data: 64 rows


,Subject_ID,Sex,Age_Group,Property,Value,Study_x,Study_y,Participant_ID,Visit_Label,C_Peptide_AUC_4Hrs
0,SUB228868,Female,13-17,GAD65,396.0,SDY1737,SDY1737,RETAIN_135342,Visit -1 (Week -1),0.942366


In [28]:
feature_columns = [
    "MIAA", "IA2IC", "GAD65", "ICA", "ZNT8",
    "Sex", "8-12", "13-17", "18-30"
]
feature_columns

['MIAA', 'IA2IC', 'GAD65', 'ICA', 'ZNT8', 'Sex', '8-12', '13-17', '18-30']

In [29]:
# Split data into train/test sets and extract X/y for ridge regression
from sklearn.model_selection import train_test_split

X_train_dict = {}
X_test_dict = {}
y_train_dict = {}
y_test_dict = {}

for study, df in merged_dfs.items():
    # Drop columns that should not be used as features
    exclude_cols = ["Study", "Subject_ID", "Participant_ID", "Visit", "C_Peptide_AUC_4Hrs"]
    feature_cols = [col for col in df.columns if col not in exclude_cols]

    # One-hot encode categorical variables (e.g., sex, age group, antibodies)
    X = pd.get_dummies(df[feature_cols], drop_first=True)
    y = df["C_Peptide_AUC_4Hrs"]

    # 80/20 train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    X_train_dict[study] = X_train
    X_test_dict[study] = X_test
    y_train_dict[study] = y_train
    y_test_dict[study] = y_test

    print(f"{study}: Train = {X_train.shape[0]}, Test = {X_test.shape[0]}")


SDY569: Train = 24, Test = 6
SDY797: Train = 196, Test = 49
SDY1737: Train = 51, Test = 13


In [30]:
def preprocess_features(df):
    df = df.copy()
    
    # Encode Sex
    if 'Sex' in df.columns:
        df['Sex'] = df['Sex'].map({'Male': 0, 'Female': 1})

    # Encode Age Group
    if 'Age Group' in df.columns:
        age_map = {'<18': 0, '18-30': 1, '>30': 2}
        df['Age Group'] = df['Age Group'].map(age_map)

    # Encode autoantibodies
    for col in df.columns:
        if 'Antibody' in col:
            df[col] = df[col].map({'Negative': 0, 'Positive': 1})

    # Ensure float32
    return df.astype('float32')


In [31]:
# Train Ridge Regression locally and evaluate with mean, median MSE and IQR
from sklearn.linear_model import Ridge
import numpy as np

local_results = {}

for study in X_train_dict:
    print(f"\nLocal Ridge Regression for {study}")
    
    X_train = X_train_dict[study].copy()
    X_test = X_test_dict[study].copy()
    y_train = y_train_dict[study].copy()
    y_test = y_test_dict[study].copy()

    # Drop rows with NaNs
    train_mask = ~(X_train.isnull().any(axis=1) | y_train.isnull())
    test_mask = ~(X_test.isnull().any(axis=1) | y_test.isnull())

    X_train = X_train[train_mask]
    y_train = y_train[train_mask]
    X_test = X_test[test_mask]
    y_test = y_test[test_mask]

    if len(X_train) == 0 or len(X_test) == 0:
        print(f"  Skipped {study} due to empty training or test set after NaN filtering.")
        continue

    # Train Ridge Regression
    model = Ridge(alpha=1.0)
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)

    # Evaluate
    squared_errors = (y_test - y_pred) ** 2
    mean_mse = np.mean(squared_errors)
    median_mse = np.median(squared_errors)
    residuals = y_test - y_pred
    iqr = np.percentile(residuals, 75) - np.percentile(residuals, 25)

    print(f"  Mean MSE:   {mean_mse:.4f}")
    print(f"  Median MSE: {median_mse:.4f}")
    print(f"  IQR:        {iqr:.4f}")

    # Store results
    local_results[study] = {
        "model": model,
        "y_test": y_test,
        "y_pred": y_pred,
        "mean_mse": mean_mse,
        "median_mse": median_mse,
        "iqr": iqr
    }




Local Ridge Regression for SDY569
  Mean MSE:   0.0491
  Median MSE: 0.0417
  IQR:        0.1678

Local Ridge Regression for SDY797
  Mean MSE:   0.0913
  Median MSE: 0.0227
  IQR:        0.2521

Local Ridge Regression for SDY1737
  Mean MSE:   0.0813
  Median MSE: 0.0444
  IQR:        0.3825


In [32]:
for study, df in merged_dfs.items():
    print(f"\nColumns in merged_dfs[{study}]:")
    print(sorted(df.columns.tolist()))



Columns in SDY569 merged data:
['Subject_ID', 'Sex', 'Age_Group', 'Property', 'Value', 'Study_x', 'Study_y', 'Participant_ID', 'Visit_Label', 'C_Peptide_AUC_4Hrs', 'Units']

Columns in SDY797 merged data:
['Subject_ID', 'Sex', 'Age_Group', 'Property', 'Value', 'Study_x', 'Study_y', 'Participant_ID', 'Visit_Label', 'C_Peptide_AUC_4Hrs', 'Units']

Columns in SDY1737 merged data:
['Subject_ID', 'Sex', 'Age_Group', 'Property', 'Value', 'Study_x', 'Study_y', 'Participant_ID', 'Visit_Label', 'C_Peptide_AUC_4Hrs']


In [33]:
# Initialize results dictionary
federated_results = {}

# Load federated model
federated_model = load_model("models/federated_3x3_model.keras")
federated_model.load_weights("models/federated_3x3.weights.h5")

# Evaluate on each study
for study in ["SDY569", "SDY797", "SDY1737"]:
    print(f"\nEvaluating {study} with federated model")

    # Extract features and labels
    X = merged_dfs[study][feature_columns]
    y = merged_dfs[study]["C_Peptide_AUC_4Hrs"]

    # Preprocess
    X_proc = preprocess_features(X)

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X_proc, y, test_size=0.33, random_state=42)

    # Skip if test set is too small
    if len(X_test) < 2:
        print(f"  Skipping {study}: not enough test data.")
        continue

    # Reshape test data for CNN
    X_test_reshaped = X_test.values.reshape(-1, 3, 3, 1)

    # Predict
    y_pred = federated_model.predict(X_test_reshaped).flatten()

    # Evaluate
    squared_errors = (y_test - y_pred) ** 2
    mean_mse = np.mean(squared_errors)
    median_mse = np.median(squared_errors)
    iqr_mse = iqr(squared_errors)

    print(f"  Mean MSE:   {mean_mse:.4f}")
    print(f"  Median MSE: {median_mse:.4f}")
    print(f"  IQR:        {iqr_mse:.4f}")

    # Store results
    federated_results[study] = {
        "model": federated_model,
        "y_test": y_test,
        "y_pred": y_pred,
        "mean_mse": mean_mse,
        "median_mse": median_mse,
        "iqr": iqr_mse
    }



Evaluating SDY569 with federated model


KeyError: "['MIAA', 'IA2IC', 'GAD65', 'ICA', 'ZNT8', '8-12', '13-17', '18-30'] not in index"